# Задание 1

[Свертка списка](https://ru.wikipedia.org/wiki/%D0%A1%D0%B2%D1%91%D1%80%D1%82%D0%BA%D0%B0_%D1%81%D0%BF%D0%B8%D1%81%D0%BA%D0%B0) -  это обобщенная операция над списком, c помощью которой можно преобразовать список в единое значение. Например, рассмотрим реализации свертки слева и свертки справа (левоассоциативную свертку и правоассоциативную свертку):

In [1]:
def foldl(f, x0, lst):
    if not lst:
        return x0
    return foldl(f, f(x0, lst[0]), lst[1:])

def foldr(f, x0, lst):
    if not lst:
        return x0
    return f(lst[0], foldr(f, x0, lst[1:]))

In [2]:
#сумма
foldl(lambda x, y: x + y, 0, [1, 2, 3])

6

In [3]:
#конкатенация
foldl(lambda x, y: '{}{}'.format(x, y), '', [1, 2, 3])

'123'

In [4]:
f = lambda x, y: x / y
foldl(f, 1, [1, 2, 3]), \
foldr(f, 1, [1, 2, 3])

(0.16666666666666666, 1.5)

Задача: реализовать foldl через foldr и наоборот. Вместо многоточий нужно вставить выражения, которые бы привели к нужному результату. Модифицировать сам список нельзя. 

In [5]:
def foldl2(f, x0, lst):
    return foldr(lambda x, p: lambda y: p(f(y, x)), lambda x: x, lst)(x0)
foldl2(lambda x, y: x + y, 0, [1, 2, 3])

6

In [6]:
def foldr2(f, x0, lst):
    return foldl(lambda p, x: lambda y: p(f(x, y)), lambda x: x, lst)(x0)
foldr(f, 1, [1, 2, 3])

1.5

# Задание 2

нужно написать функцию, которая принимает две строки и проверяет, входит ли хотя бы одна перестановка второй строки в первую. Например:

> a = 'abcrotm'
> 
> b = 'tro'

функция def check_inv(a, b) вернет True, так как 'rot' содержится в 'abcrotm'. Нужно подумать как можно более оптимальный алгоритм и оценить его сложность. 

In [7]:
from collections import Counter

a = 'abcrotm'
b = 'tro'
def check_inv(a, b):
    n_a = len(a)
    n_b = len(b)
    count_a = Counter(a[:n_b - 1])
    for key in Counter(a).keys():
        if key not in count_a.keys():
            count_a[key] = 0
    count_b = Counter(b)
    for key in Counter(a).keys():
        if key not in count_b.keys():
            count_b[key] = 0
    for i in range(n_b - 1, n_a):
        count_a[a[i]] += 1
        if count_a == count_b:
            ans = True
            break
        count_a[a[i + 1 - n_b]] -= 1
        
    if ans != True:
        ans = False
    return ans

In [8]:
check_inv(a, b)

True

# Задание 3

Реализовать бинарное дерево (класс Tree), в нём методы __repr__, __str__, __iter__ (итерация только по листьям).

In [9]:
class Tree:
    def __init__(self, value=None, left=None, right=None):
        self.left = left
        self.right = right
        self.value = value
        
    def __iter__(self):
        ans_l = []
        ans_r = []
        
        if self.left:
            ans_l = list(self.left)
        if self.right:
            ans_r = list(self.right)
        if self.left == None and self.right == None:
            ans = iter([self.value])
            return ans
        ans = iter(ans_l + ans_r)
        return ans
        

    def __str__(self):
        if self.left == None and self.right == None:
            ans = str(self.value)
            return ans
        ans = []
        if self.left:
            ans.append(self.left.__str__())
        if self.right:
            ans.append(self.right.__str__())          
        return ' '.join(ans)
    
    def __repr__(self): #using f-strings
        
        if self.left is None and self.right is None:
            return f"Tree({self.value})"
        elif self.left == None:
            return f"Tree({self.value}, {self.right.__repr__()})"
        elif self.right == None:
            return f"Tree({self.value}, {self.left.__repr__()})"       
        else:
            return f"Tree({self.value}, {self.left.__repr__()}, {self.right.__repr__()})" 
 
    
tree = Tree(0, Tree(1, Tree(3), Tree(4)), Tree(2))

print(list(tree) == [3, 4, 2])
print(tree)

True
3 4 2


# Задание 4

Реализовать простейший калькулятор математических выражений:
- только целые числа
- **+**, **\-**, **\***, **\**
- скобки

**Можно использовать регулярные выражения**

In [10]:
def split(expr):
    result = []
    expr = expr.split()
    for value in expr:
        if(value[0] == '('):
            result.append('(')
            result.append(value[1:])
        elif(value[-1] == ')'):
            result.append(value[:-1])
            result.append(')')
        else:
            result.append(value)
    return result

def calc(expr):
    expr = split(expr)
    
    operations = {
        '+': lambda a, b: a + b,
        '-': lambda a, b: a - b,
        '*': lambda a, b: a * b,
        '/': lambda a, b: a / b}

    total = []
    operands = []

    for value in expr:
        if value == '(':
            operands.append(value)
        if value == ')':
            while operands[-1] != '(':
                total.append(operands.pop()) #добавляем все, что между скобками и удаляем скобки
            operands.pop()
        if value == '*' or value == '/':
            operands.append(value)
        if value == '+':
            while len(operands) > 0 and (operands[-1] == '*' or operands[-1] == '/' or operands[-1] == '-'): #тк умножение и деление более приоритетны, а разность = сумме (по приоритету)
                total.append(operands.pop())
            operands.append(value)            
        if value == '-':
            while len(operands) > 0 and (operands[-1] == '*' or operands[-1] == '/' or operands[-1] == '+'): #аналогично
                total.append(operands.pop())
            operands.append(value)  
        if value.isnumeric() == True:
            total.append(value)
            
    operands.reverse()
    for i in operands:
        total.append(i)
    
    result = []
    
    for value in total:
        if value.isnumeric() == True:
            result.append(int(value))
        if value in operations.keys():
            func = operations[value]
            temp = result
            result = result[:-2]
            result.append(func(temp[-2], temp[-1]))
            
    ans = result[0]
    return ans

In [11]:
calc('2 * (15 - 3 * 4) - 2') == 4

True

In [12]:
calc('10 - 5 * 3 + (2 + 6 * 1) / 4')

-3.0